# Classificação de Faces

A rede neural artificial aqui utilizada é uma variante da arquitetura inception. Uma variante NN4 do projeto OpenFace. A implementação usa uma implementação Keras do modelo pego do projeto Keras-OpenFace. 

In [3]:
from model import create_model

nn4_small2 = create_model()

Instructions for updating:
Colocations handled automatically by placer.


O treinamento do modelo visa aprender um $f(x)$ incorporado a uma imagem $x$, cuja distância quadrada L2 entre todas as faces da mesma identidade é pequena e a distância entre os pares de faces de diferentes identidades é grande. Isso pode ser atingido através do _triplet loss_ L que é minimizado quando a distância entre uma imagem âncora $x_i^a$ e uma imagem positiva $x_i^p$(de mesma identidade) no espaço é menor que a distância entre aquela imagem âncora e uma imagem negativa $x_i^n$ em pelo menos uma margem $\alpha$.

$\begin{aligned}
L = \sum_{i = 1}^{m}{[\lVert f(x_i^a) - f(x_i^p) \rVert_2^2 - \lVert f(x_i^a) - f(x_i^n) \rVert_2^2 + \alpha]}_+
\end{aligned}$

In [4]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

# Entrada para imagens âncora, postiva e negativa
in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))

# Sáida para vetores embutidos âncora, positivo e negativo
# o modelo de instâncias nn4_small é compartilhado (Redes Siamesas)
emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Camada que computa o triplet loss para os vetores embutidos âncora, positivo e negativo
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Modelo que pode ser treinado com imagens âncora, positiva e negativa
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)
